In [1]:
import spacy
import datetime
import nltk
import re
from app.sql_conn import *

In [2]:
brands = sql_brands()

df = sql_df('nos')
df

,data,title,comment,link,created_at
id,,,,,
1,9 de outubro 2020,Mau atendimento cinema,venho por este meio apresentar a minha indigna...,https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:45:55
2,5 de janeiro 2019,Faturação,"Após me deslocar à Nós do Forum Montijo ,no di...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:45:58
3,28 de setembro 2021 |Data de ocorrência: 28 de...,Cobrança indevida,"Ao contratar os serviços de Internet, pedi à o...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:46:01
4,4 de julho 2017,Mau Serviço - Suspensão serviço móvel,"Boa noite, Na sequência da queixa anterior, po...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:46:04
5,5 de agosto 2015,Incumprimento contratual,"No dia 18 de Maio de 2015, uma colaboradora da...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:46:07
...,...,...,...,...,...
19119,8 de setembro 2017,AUSENCIA DE CONTATOS E ESCLARECIMENTOS,Estive na Loja de oeiras para dar baixa de ser...,https://portaldaqueixa.com/brands/nos/complain...,2022-01-24 17:59:36
19120,14 de junho 2019 (editada a 18 de junho 2019),Velocidade de internet não corresponde a contr...,"Olá, Venho por este meio reclamar do mau servi...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-24 17:59:38
19121,18 de fevereiro 2021,Reclamação - cobrança indevida,Sou cliente da NOS há alguns anos e aderi a um...,https://portaldaqueixa.com/brands/nos/complain...,2022-01-24 17:59:41


In [3]:
def date_parser(date_string):
    """convert dates have convert """

    mapping = {'janeiro': '01', 'fevereiro': '02', 'março': '03',
               'abril': '04', 'maio': '05', 'junho': '06',
               'julho': '07', 'agosto': '08', 'setembro': '09',
               'outubro': '10', 'novembro': '11', 'dezembro': '12', 'zembro': '12'}

    date_info = re.sub("\(.*?\)", '', date_string)
    date_info = date_info.lower().replace('de', '').split()
    try:
        day, month_pt, year = date_info
        month = mapping[month_pt]
        date_iso = '{}-{:02d}-{:02d}'.format(year, int(month), int(day))
        date_object = datetime.datetime.strptime(date_iso, '%Y-%m-%d')

        return date_object

    except:
        pass

In [4]:
df['data'] = df['data'].map(date_parser, na_action='ignore')
df = df.dropna()
df

,data,title,comment,link,created_at
id,,,,,
1,2020-10-09,Mau atendimento cinema,venho por este meio apresentar a minha indigna...,https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:45:55
2,2019-01-05,Faturação,"Após me deslocar à Nós do Forum Montijo ,no di...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:45:58
4,2017-07-04,Mau Serviço - Suspensão serviço móvel,"Boa noite, Na sequência da queixa anterior, po...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:46:04
5,2015-08-05,Incumprimento contratual,"No dia 18 de Maio de 2015, uma colaboradora da...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:46:07
6,2020-08-20,Telefonemas de marketing abusivo,"Nos últimos dias, tenho recebido uma série de ...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:46:10
...,...,...,...,...,...
19119,2017-09-08,AUSENCIA DE CONTATOS E ESCLARECIMENTOS,Estive na Loja de oeiras para dar baixa de ser...,https://portaldaqueixa.com/brands/nos/complain...,2022-01-24 17:59:36
19120,2019-06-14,Velocidade de internet não corresponde a contr...,"Olá, Venho por este meio reclamar do mau servi...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-24 17:59:38
19121,2021-02-18,Reclamação - cobrança indevida,Sou cliente da NOS há alguns anos e aderi a um...,https://portaldaqueixa.com/brands/nos/complain...,2022-01-24 17:59:41


In [5]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
new_stopwords = ['tarde', 'venho', 'noite', 'dia', 'ja', 'pois', 'ser', 'ter', 'tendo', 'nao', 'destas', 'têm',
                 'neste',
                 'porque', 'fiz', 'janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto',
                 'setembro',
                 'outubro', 'novembro', 'dezembro', 'ate', 'talvez', 'aqui', 'portanto', 'estar', 'iria', 'ia',
                 'eis', 'vem', 'ir', 'preciso', 'sido', 'acerca', 'seguinte', 'domingos', ]
stopwords.extend(new_stopwords)
stopwords.extend(['meo'])
print(stopwords)

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estiv

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df['comment'] = df['comment'].str.lower()

C:\Users\joaom\AppData\Local\Temp\ipykernel_13668\2875613913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].str.lower()


In [7]:
nlp = spacy.load('pt_core_news_sm', disable=['parser', 'ner'])

def lemma(text):

    doc = nlp(text)
    result = " ".join([token.lemma_ for token in doc])

    return result

In [8]:
df['lemma'] = df['comment'].map(lemma)

C:\Users\joaom\AppData\Local\Temp\ipykernel_13668\1508308703.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemma'] = df['comment'].map(lemma)


In [9]:
df

,data,title,comment,link,created_at,lemma
id,,,,,,
1,2020-10-09,Mau atendimento cinema,venho por este meio apresentar a minha indigna...,https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:45:55,vir por este mear apresentar o meu indignação ...
2,2019-01-05,Faturação,"após me deslocar à nós do forum montijo ,no di...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:45:58,"após me deslocar à nó do forum montijo , o dia..."
4,2017-07-04,Mau Serviço - Suspensão serviço móvel,"boa noite, na sequência da queixa anterior, po...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:46:04,"bom noite , o sequência da queixar anterior , ..."
5,2015-08-05,Incumprimento contratual,"no dia 18 de maio de 2015, uma colaboradora da...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:46:07,"o dia 18 de maio de 2015 , umar colaborador da..."
6,2020-08-20,Telefonemas de marketing abusivo,"nos últimos dias, tenho recebido uma série de ...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-21 14:46:10,"o último dia , ter receber umar série de telef..."
...,...,...,...,...,...,...
19119,2017-09-08,AUSENCIA DE CONTATOS E ESCLARECIMENTOS,estive na loja de oeiras para dar baixa de ser...,https://portaldaqueixa.com/brands/nos/complain...,2022-01-24 17:59:36,estivar o loja de oeiras parir dar baixo de se...
19120,2019-06-14,Velocidade de internet não corresponde a contr...,"olá, venho por este meio reclamar do mau servi...",https://portaldaqueixa.com/brands/nos/complain...,2022-01-24 17:59:38,"olá , vir por este mear reclamar do mau serviç..."
19121,2021-02-18,Reclamação - cobrança indevida,sou cliente da nos há alguns anos e aderi a um...,https://portaldaqueixa.com/brands/nos/complain...,2022-01-24 17:59:41,ser cliente da o haver algum ano e aderir o um...


In [14]:
df['tokens'] = df['lemma'].map(lambda token: [t for t in token if t.lower() not in stopwords])

C:\Users\joaom\AppData\Local\Temp\ipykernel_12764\2417289696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['lemma'].apply(lambda token: [t for t in token if t.lower() not in stopwords])
